# Ridge Regression

# 1. Preprocessing Data

In [1]:
# Import the libraries
import pandas as pd 
import numpy as np
import re 

In [2]:
# Processing data txt 
with open("/Users/charles/MLGT/SESSION 1/Data/DeathRate.txt") as f:
    # Read data, split each line, store in list
    raw_data_lines = f.read().splitlines()
    lines = []
    for i in range(len(raw_data_lines)):
        # Delete some space in each line with regex (raw)
        line = re.sub(" +", " ", raw_data_lines[i])
        # Delete first row
        line = line.split()[1:]
        lines.append(line)

In [3]:
# Assign info for first row
init_col = np.append(["A" + str(i) for i in range(1,16)], "B")
# Covert to DF 
data = pd.DataFrame(lines, columns=init_col)
# Change index (first col)
data.index = [i for i in range(1, len(data)+1)]

In [4]:
# Convert data to numeric 
data = data.apply(pd.to_numeric)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 1 to 60
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A1      60 non-null     int64  
 1   A2      60 non-null     int64  
 2   A3      60 non-null     int64  
 3   A4      60 non-null     float64
 4   A5      60 non-null     float64
 5   A6      60 non-null     float64
 6   A7      60 non-null     float64
 7   A8      60 non-null     int64  
 8   A9      60 non-null     float64
 9   A10     60 non-null     float64
 10  A11     60 non-null     float64
 11  A12     60 non-null     int64  
 12  A13     60 non-null     int64  
 13  A14     60 non-null     int64  
 14  A15     60 non-null     int64  
 15  B       60 non-null     float64
dtypes: float64(8), int64(8)
memory usage: 8.0 KB


In [5]:
data.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,B
1,36,27,71,8.1,3.34,11.4,81.5,3243,8.8,42.6,11.7,21,15,59,59,921.870
2,35,23,72,11.1,3.14,11.0,78.8,4281,3.6,50.7,14.4,8,10,39,57,997.875
3,44,29,74,10.4,3.21,9.8,81.6,4260,0.8,39.4,12.4,6,6,33,54,962.354
4,47,45,79,6.5,3.41,11.1,77.5,3125,27.1,50.2,20.6,18,8,24,56,982.291
5,43,35,77,7.6,3.44,9.6,84.6,6441,24.4,43.7,14.3,43,38,206,55,1071.289


# 2. Normalize Data

In [6]:
def normalize_and_add_ones(X):
    X = np.array(X)
    # Normalize MinMaxScaler (0,1), search in each row to find min/max element in col
    X_min = np.array([[np.amin(X[:, id_col]) 
                        for id_col in range(X.shape[1])]     # Check all col in each row
                            for _ in range(X.shape[0])])     # Loop all rows
    X_max = np.array([[np.amax(X[:, id_col]) 
                        for id_col in range(X.shape[1])]     # Check all col in each row
                            for _ in range(X.shape[0])])     # Loop all rows
    X_normalized = (X - X_min) / (X_max - X_min)

    # X_bias with 1 for first column
    X_bias = np.concatenate((np.ones((X_normalized.shape[0], 1)), X_normalized), axis = 1)
    return X_bias

In [ ]:
# Apply normalize & add column bias A0
data = normalize_and_add_ones(data)
init_col = np.insert(init_col, 0, "A0")    # Run only one 
data = pd.DataFrame(data, columns=init_col)

In [9]:
data.head()

,A0,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,B
0,1.0,0.52,0.211268,0.363636,0.403226,0.688525,0.727273,0.615063,0.218213,0.212202,0.339768,0.135294,0.030912,0.044025,0.209386,0.600000,0.406723
1,1.0,0.50,0.154930,0.409091,0.887097,0.360656,0.606061,0.502092,0.343909,0.074271,0.652510,0.294118,0.010819,0.028302,0.137184,0.542857,0.642454
2,1.0,0.68,0.239437,0.500000,0.774194,0.475410,0.242424,0.619247,0.341366,0.000000,0.216216,0.176471,0.007728,0.015723,0.115523,0.457143,0.532285
3,1.0,0.74,0.464789,0.727273,0.145161,0.803279,0.636364,0.447699,0.203923,0.697613,0.633205,0.658824,0.026275,0.022013,0.083032,0.514286,0.594120
4,1.0,0.66,0.323944,0.636364,0.322581,0.852459,0.181818,0.744770,0.605473,0.625995,0.382239,0.288235,0.064915,0.116352,0.740072,0.485714,0.870149


# 3. Implement class Ridge Regression

In [ ]:
class RidgeRegression:
    # Init class with self
    def __init__(self):
        return

    # fit with mini-batch GD with some epochs
    def fit(self, X_train, y_train, learning_rate, lambda, max_epoch = 10, batch_size = 128):
        # Init W with sample standard normal distribution
        W = np.random.randn(X_train.shape[0])

        # Init last_lost for check to stop GD
        last_loss = 10e+8

        # Run for each epoch 
        for epoch in range(max_epoch):
            # Create list index col of X_train 
            arr = np.array(range(X_train.shape[0]))
            # Shuffle element of array with formula arr[[indices]]
            np.random.shuffle(arr)
            # Random row train & test
            X_train = X_train[arr]
            y_train = y_train[arr]
            # Size of mini-batch
            total_minibatch = int(np.ceil(X_train.shape[0] / batch_size))

            # Run epoch with mini-batch (shuffled)
            for i in range(total_minibatch):
                index = i * batch_size
                X_train_sub = X_train[index: index+batch_size]
                y_train_sub = y_train[index: index+batch_size]
                # GD
                grad = X_train_sub.T.dot(X_train_sub.dot(W) - y_train_sub) + lambda * W
                W = W - learning_rate * grad            
            # Check new loss, if <<, then break loop
            new_loss = self.rss(self.predict(W, X_new), y_train)
            if (np.abs(new_loss - last_loss) <= 1e-5):
                break
            last_lost = new_loss
        return W

    # Predict with formula Regression Y = X.W
    def predict(self, W, X_new):
        X_new = np.array(X_new)
        y_new = X_new.dot(W)
        return y_new
    
    # RSS
    def rss(self, y_new, y_predict):
        # y_predict - y_train = vector, then sum of all ^2 for all elements -> R
        return np.sum((y_new - y_predict)) / (y_new.shape[0]_

    # Cross-validation (Run first)
    def find_best_lambda():
        


![](2022-02-25-22-34-21.png)